## Hands-on session 2.3 - CRM basics with ReMKiT1D

In this session we cover the basics of CRM construction for use in ReMKiT1D models. The example is focused on a small set of equations, in order to demonstrate the general concepts. 

Demonstrated concepts:

- Setting species and associating variables to them 
- Simple and derived transitions 
- Constructing CRM modelbound data from transitions
- Adding term generators that use CRM modelbound data
- The extractor manipulator

A number of CRM concepts is beyond this workshop. These include:

- Kinetic features - Boltzmann term generators 
- Built-in Janev hydrogen transitions 
- Using external databases such as AMJUEL to construct polynomial fits 
- Other kinds of transitions (detailed balance etc.)

In [1]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds 
import RMK_support.crm_support as crm

import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for writing the files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/day_2_3/"
rk.setHDF5Path(hdf5Filepath) 

### Grid initialization

We initialize 0D grid as in the previous session

In [ ]:
rk.grid = Grid(np.zeros(1))

### A simplified Collisional Radiative Model 

Let's write a simplified nonlinear time-dependent CRM. 

Let's assume we're working with electrons, singly-charged ions, a ground state and an excited neutral state. This implies that we are tracking the following densities:

$$n_e,n_i,n_1,n_2$$

Let's then say we have the following reactions in the system:

- Electron-impact ionization of both states 

$$ e + b \rightarrow i + e + e,\quad b=1,2$$  

- The direct inverse of the above reaction - three-body recombination

$$ i + e + e \rightarrow b + e,\quad b=1,2$$  

- Electron impact excitation from state 1 to state 2

$$ e + 1 \rightarrow e + 2$$  

- Radiative de-excitation (spontaneous emission) from state 2 to state 1 

$$ n_2 \rightarrow n_1 + h\nu$$ 
where $h\nu$ signifies a photon with the transition energy. We will assume that any energy losses are recuperated through some heating of the electrons, maintaining a constant electron temperature. This is assumption can be relaxed, but required the evolution of electron energy and unduly complicated this system.

The system described above can be represented with the following system of nonlinear ODEs

$$ \frac{d n_1}{dt} = -K_{12}n_en_1 - K_1^{ion}n_en_1 + A_{21}n_2 + K_1^{rec}n_e^2n_i $$

$$ \frac{d n_2}{dt} = K_{12}n_en_1 - K_2^{ion}n_en_2 - A_{21}n_2 + K_2^{rec}n_e^2n_i $$
$$ \frac{d n_e}{dt} = \frac{d n_i}{dt} =  K_1^{ion}n_en_1 + K_2^{ion}n_en_2 - (K_1^{rec} + K_2^{rec})n_e^2n_i$$

In this case, one could simplify the system by dropping the ion density equation, but in we might want to track many different ionization states so $n_e \neq n_i$ in general.

Even though we have only 6 reactions, the number of terms (ignoring potentially simplifying groupings) in this system is 16. The CRM features in ReMKiT1D are meant to simplify the building of many related terms, such as these. 

### Species information in ReMKiT1D

Many built-in terms and models, including those generated with CRM term generators, require knowledge of the species they are being constructed for. This includes both basic information about the species (such as atomic mass and charge), as well as any associated variables and unique IDs.

**NOTE**: ReMKiT1D enforces that the electron species, with name "e" has the speciesID of 0. Other than that, the convention so far in ReMKiT1D has been that non-electron charged species get negative indices and neutral species get positive indices. 

Let's add the species present in the toy model we just made. Note that the mass and charge values are not used by the CRM data, but we include them for completeness.

In [ ]:
rk.addSpecies(name="e",speciesID=0,associatedVars=["ne"]) # ReMKiT1D will detect the electron species automatically and populate the mass and charge fields
rk.addSpecies(name="i",speciesID=-1,atomicA=1.0,charge=1,associatedVars=["ni"])
rk.addSpecies(name="1",speciesID=1,atomicA=1.0,charge=0,associatedVars=["n1"])
rk.addSpecies(name="2",speciesID=2,atomicA=1.0,charge=0,associatedVars=["n2"])